<center><img src=img/MScAI_brand.png width=70%></center>

# $n$-grams, randomness, and the Aaronson oracle

The Aaronson oracle is about the simplest possible computer game, but it really makes you think. We'll describe it and see how to get the computer to play it using a very broadly applicable technique, $n$-grams. We'll start with motivation.

### Which of these is random?

 <table style="width:100%">
  <tr>
    <td><img src=img/randomness_pointa.jpg width=80%></td>
    <td><img src=img/randomness_pointb.jpg width=80%></td>
  </tr>
</table> 

<font size=1>From <a href=https://telescoper.wordpress.com/2009/04/04/points-and-poisson-davril/>telescoper</a> via <a href="http://blogs.discovermagazine.com/cosmicvariance/2009/04/06/perceiving-randomness/#.XcLxeNHLfeR">Discover magazine</a>

The left-hand image is random. The right-hand one is too evenly-spaced to be truly random.

It turns out that humans are very bad at detecting randomness. We're so good at detecting patterns that we detect them even when they're not there!

The same thing could happen in other settings. Which of these is random?

* 10011000110100011000111010100010110000110001111110
* 01001010010101010001001001011010101101010110101100

The [Aaronson oracle](https://github.com/elsehow/aaronson-oracle) is a game invented by Scott Aaronson (a top researcher in computational complexity and quantum computing, and he has a [great blog](https://www.scottaaronson.com/blog/)) to probe the idea of randomness.


* The human types '0' or '1' (or 'f' or 'd', or left-arrow or right-arrow, whatever) -- should be fairly quick.
* The computer has made a guess which it will be. 
* We repeat many times (the human should type fairly quickly). The computer has access to the history.
* If the computer's accuracy is near 50%, the human wins. If high (e.g. 65%), the computer wins.

The program is just a loop which makes a prediction (let's just predict randomly for now), reads in the user's move, saves both to a history, and calculates a running accuracy. 

```python
    move_hist = []
    pred_hist = []
    while True:
        # generate a prediction, just placeholder
        prediction = random.choice("01")

        # read user's move
        user_move = getch()

        # store to history
        move_hist.append(user_move)
        pred_hist.append(prediction)
        
        # stats
        accuracy = np.mean([p == u for p, u in 
                    zip(pred_hist, move_hist)])
        print(f"prediction: {prediction}; " +
              f"user played {user_move}; " +
              f"accuracy {accuracy}")        
```

### `input` and `getch`

In Python, there is a function `input()` (a component of the read-eval-print loop!) which gets a line from the user.

But that won't work here because the user would have to type 0 `<return>` or 1 `<return>` each time which would be slow and could change their behaviour.

Instead, there is a small module `code/getchar.py` which reads one character at a time, available here:

* https://stackoverflow.com/q/510357/86465
* http://code.activestate.com/recipes/134892/




Our remaining task is to implement a method of predicting the user's move. There are many possible approaches, each of which makes some implicit assumptions about the patterns that will be present despite the user's intentions. We'll use a very simple method which performs amazingly well: $n$-grams.

* Suppose we observe that whenever the user types `01010` they usually type `1` next. That means that whenever we observe `01010` we should predict `1`.

### $n$-grams

An $n$-gram is an $n$-tuple of consecutive objects drawn from a sequence. For example in the sentence "it was the best of times, it was the worst of times", we have the following 3-grams:

(it was the), (was the best), (the best of), (best of times), etc.

$n$-grams have applications in language modelling and generation, both for natural language and formal languages, and even the "language" of music.

Here, we will use $n$-grams to make predictions. Let's choose $n=5$. For every possible 5-gram, we'll track how often it is followed by a `0` and how often by a `1`.

Suppose we have the string `010101001`. Then we get these observations:

```
01010 -> 1
10101 -> 0
01010 -> 0
10100 -> 1
```

We'll represent each observation as a 2-tuple, e.g. `("01010", "1")`. We'll just put the tuples in a `Counter`, which is a specialised `defaultdict`.

```python
from collections import Counter
c = Counter()
c["01010", "1"] += 1
c["10101", "0"] += 1
```
etc.

Then whenever we observe a string `"01010"`, we just check which has the highest frequency in the data so far: `("01010", "0")`, or `("01010", "1")`.

Amazingly, this is good enough to perform quite well! The full program is in `code/aaronson_oracle.py`:

```
$ cd code
$ python aaronson_oracle.py
```

### Exercises

* Play the game a few times and see how you get on. Remember, you're supposed to play quickly.
* Tell it to read the $n$-grams from a previously-saved file if it exists, and to write an updated version at the end,  e.g. `python aaronson_oracle.py ngrams.txt`.
* Look at the Counter that is saved in `ngrams.txt` and see if you can see your own weakness.
* Change $n$ and see if it gets better or worse.

### Exercises (only for fun):

1. Write an ML program using Scikit-Learn which can perform better than the n-gram program, ie better than 60% against a typical human.
2. Write a deterministic player which can defeat the program, ie keep it close to 50%.
3. Try playing your deterministic player (2) against your friend's enhanced predictor (1).
4. Try playing against your friends manually.